### 迭代器

以直接作用于for循环的数据类型有以下几种：

一类是集合数据类型（list、tuple、dict、set、str）,一类是generator包括生成器和yield，这些可以直接作用于for循环的对象都统称为可迭代对象。

#### 判断是否可迭代

可用isinstance()判断一个对象是否是 Iterable对象：

In [1]:
from collections import Iterable
isinstance("abc",Iterable)  #True,可迭代
isinstance(100,Iterable)    #False,不可迭代

False

#### 可以被next()函数调用并不断返回下一个值的对象称为迭代器：Iterator.可以使用isinstance()判断一个对象是否是Iterator对象：

In [5]:
from collections import Iterator
isinstance((x for x in range(10)),Iterator)

True

In [6]:
from collections import Iterator
isinstance([],Iterator)

False

#### 生成器都是Iterator,但list、dict/str虽然是Iteratable,却不是Iterator。把list、dict、str等Iterable可以使用iter()函数

In [8]:
isinstance(iter([]),Iterator)

True

In [9]:
isinstance(iter("abc"),Iterator)

True

#### 总结：凡是可作用于for循环的对象都是 Iterable类型；
   ####     凡是作用于next()的函数的对象都是Iterator类型；
####            集合数据类型如list、dict、str等是Iterable 但不是 Iterator，不过可以通过iter()函数获得一个Iterator对象。

### 闭包

在函数内部再定义一个函数，并且这个函数用到了外边函数的变量，那么我们将这个函数以及用到的一些变量，里面的整体为闭包

In [10]:
#定义一个函数
def test(number):
    
    #在函数内部再定义一个函数，并且这个函数用到了外边函数的变量，那么我们将这个函数以及用到的一些变量，里面的整体为闭包
    def test_in(number_in):
        print("in test_in 函数，number_in is %d" % number_in)
        return number+number_in
    
    #其实这里返回的就是闭包的结果
    return test_in                  #####return了 test_in 指向的函数体

#给test函数赋值，这个20就是给参数number
ret = test(20)

#注意这里的100其实给参数number_in
print(ret(100))

#注意这里的200其实给参数number_in\
print(ret(200))

in test_in 函数，number_in is 100
120
in test_in 函数，number_in is 200
220


In [11]:
#闭包特点:当执行test(100)返回number为100，再次执行ret()会使用上一次的number值最后得到结果101
def test(number):
    print("---1---")
    def test_in(number2):
        print("---2---")
        print(number+number2)
    print("---3---")
    return test_in   #返回test_in指向test_in()函数的引用

ret = test(100) #ret指向test_in，也就是全部指向test_in()的函数体
print("-"*10)
ret(1)

---1---
---3---
----------
---2---
101


#### 闭包的应用

1、简化调用方式，比传统写法调用简便（a,b不变，只变化x）

In [12]:
def line_conf(a, b):
    
    def line(x):
        return a*x + b
    
    return line

line1 = line_conf(1,2)
line2 = line_conf(4,5)
print(line1(5))
print(line2(5))

7
25


### 装饰器

在不改变原函数功能的情况下添加新功能

In [16]:
def w1(func):
    def inner():
        print("- - -正在验证权限- - -")
        if True:
            func()
        else:
            print("没有权限")
    return inner

@w1    
def f1():
    print("- - -f1- - -")

@w1
def f2():
    print("- - -f2- - -")
    
# innerFunc = w1(f1)
# innerFunc()

#f1 = w1(f1)  # 等价于 @w1
f1()
f2()

- - -正在验证权限- - -
- - -f1- - -
- - -正在验证权限- - -
- - -f2- - -


### 2个装饰器

In [4]:
#定义函数：完成包裹数据
def makeBold(fn):
    def wrapped():
        print("----1---")
        return "<b>" + fn() + "</b>"
    return wrapped

#定义函数：完成包裹数据
def makeItalic(fn):
    def wrapped():
        print("----2---")
        return "<i>" + fn() + "</i>"
    return wrapped

#先执行装饰器@makeItalic
@makeBold     #test3 = makeBold(test3)
@makeItalic   #test3 = makeItalic(test3)
def test3():
    print("----3---")
    return "hello world-3"

ret = test3()
print(ret)

#执行流程:
#1.@makeBold先执行，打印 print("----1---")，return wrapped函数，fn()传入调用@makeItalic
#2.@makeItalic继续执行，打印 print("----2---")，return wrapped函数，fn()传入调用test3()，输出 print("----3---")和<i>hello world-3</i>
#3.最后@makeBold才调用@makeItalic的return值

----1---
----2---
----3---
<b><i>hello world-3</i></b>


### 装饰器执行的时间

In [6]:
def w1(func):
    print("---正在装饰1----")
    def inner():
        print("---正在验证权限1----")
        func()
    return inner

def w2(func):
    print("---正在装饰2----")
    def inner():
        print("---正在验证权限2----")
        func()
    return inner

#只要python解释器执行到了这个代码,那么就会自动的进行装饰,而不是等到调用的时候才装饰的
@w1
@w2
def f1():
    print("---f1---")

#在调用f1之前,已经进行装饰了
f1()

---正在装饰2----
---正在装饰1----
---正在验证权限1----
---正在验证权限2----
---f1---


### 装饰器对有参数函数进行装饰

In [4]:
#不带参数的装饰器
def func(functionName):
    print("- - -func- - - 1 - -")
    def func_in():
        print("- - -func_in - - 1 - -")
        functionName()
        print("- - -func_in - - 2 - -")
    print("- - -func- - - 2 - -")
    return func_in

@func
def test():
    print("- - -test- - -")
    
# test = func(test)
test()

- - -func- - - 1 - -
- - -func- - - 2 - -
- - -func_in - - 1 - -
- - -test- - -
- - -func_in - - 2 - -


In [9]:
#定长参数
def func(functionName):
    print("- - -func- - - 1 - -")
    def func_in(a,b):  #如果a,b 没有定义,那么会导致16行的调用失败
        print("- - -func_in - - 1 - -")
        functionName(a,b)#如果没有把a,b当做实参进行传递,那么会导致调用12行的函数失败
        print("- - -func_in - - 2 - -")
    print("- - -func- - - 2 - -")
    return func_in

@func
def test(a,b):
    print("- - -test-a=%d,b=%d- - -"%(a,b))

test(11,22)

- - -func- - - 1 - -
- - -func- - - 2 - -
- - -func_in - - 1 - -
- - -test-a=11,b=22- - -
- - -func_in - - 2 - -


In [12]:
#不定长参数
def func(functionName):
    print("- - -func- - - 1 - -")
    def func_in(*args,**kwargs):#采用不定长参数的方式满足所有函数需要参数以及不需要参数的情况
        print("- - -func_in - - 1 - -")
        functionName(*args,**kwargs)#这个地方,需要写*以及**,如果不写的话,那么args是元祖,而kwargs是字典
        print("- - -func_in - - 2 - -")
    print("- - -func- - - 2 - -")
    return func_in

@func
def test(a,b,c):
    print("- - -test-a=%d,b=%d,c=%d- - -"%(a,b,c))
    
@func
def test2(a,b,c,d):
    print("- - -test-a=%d,b=%d,c=%d,d=%d- - -"%(a,b,c,d))
    
test(11,22,33)
test2(44,55,66,77)

- - -func- - - 1 - -
- - -func- - - 2 - -
- - -func- - - 1 - -
- - -func- - - 2 - -
- - -func_in - - 1 - -
- - -test-a=11,b=22,c=33- - -
- - -func_in - - 2 - -
- - -func_in - - 1 - -
- - -test-a=44,b=55,c=66,d=77- - -
- - -func_in - - 2 - -


### 装饰器对带有返回值的函数进行装饰

In [14]:
#返回值为空
def func(functionName):
    print("- - -func- - - 1 - -")
    def func_in():
        print("- - -func_in - - 1 - -")
        functionName()
        print("- - -func_in - - 2 - -")
    print("- - -func- - - 2 - -")
    return func_in

@func 
def test():
    print("- - -test- - -")
    return "Hello world"

ret = test()
print("test return value is %s"%ret)

- - -func- - - 1 - -
- - -func- - - 2 - -
- - -func_in - - 1 - -
- - -test- - -
- - -func_in - - 2 - -
test return value is None


In [21]:
#有返回值
def func(functionName):
    print("- - -func- - - 1 - -")
    def func_in():
        print("- - -func_in - - 1 - -")
        ret = functionName() #保存返回来的 Hello world
        print("- - -func_in - - 2 - -")
        return ret  #把 Helloworld 返回到 ret=test()处的调用
    print("- - -func- - - 2 - -")
    return func_in

@func 
def test():
    print("- - -test- - -")
    return "Hello world"

ret = test()
print("test return value is %s"%ret)
### ret = test() 中的 test()函数指向func_in(),去调func_in()函数，当func_in()函数无返回值时，test()函数就没有返回值，即为None
### functionName指向 def test()这个函数，故执行完functionName()后，functionName得到了返回值 Hello world，此时找个变量接收functionName()
#   执行后的返回值("Hello world"),在return，就可以让func_in()函数有返回值，即ret=test()中的test()执行后有返回值

- - -func- - - 1 - -
- - -func- - - 2 - -
- - -func_in - - 1 - -
- - -test- - -
- - -func_in - - 2 - -
test return value is Hello world


### 通用装饰器

In [20]:
def func(functionName):
    def func_in(*args,**kwargs):
        print("- - -记录日志- - -")
        ret = functionName(*args,**kwargs)
        return ret
    
    return func_in

@func 
def test():
    print("- - -test- - -")
    return "Hello world"

@func
def test2():
    print("- - -test2- - -")
    
@func
def test3(a):
    print("- - - test3-a=%d - - -"%a)

ret = test()
print("test return value is %s"%ret)

test2()

test3(11)

- - -记录日志- - -
- - -test- - -
test return value is Hello world
- - -记录日志- - -
- - -test2- - -
- - -记录日志- - -
- - - test3-a=11 - - -


### 带有参数的装饰器